In [1]:
import numpy as np
import pandas as pd

In [2]:
def plot_regression(f, df):
    models = sorted(set(df["model"]))
    all_factors = sorted(set(df["factor"]))
    for mi, m in enumerate(models):
        dfm = df[df["model"] == m]
        low = min(dfm["beta"] - 2.0*dfm["se"])
        high = max(dfm["beta"] + 2.0*dfm["se"])
        most = max(high, -low)
        x = []
        y = []
        xerr = []
        xlab = []
        for i, factor in enumerate(all_factors):
            xlab.append(factor)
            y.append(i)
            try:
                dfmf = dfm[dfm["factor"] == factor]
                x.append(list(dfmf["beta"])[0])
                xerr.append(list(dfmf["se"])[0])
            except IndexError:
                x.append(2.0*most) # Hide point outside chart
                xerr.append(0)
        subplot(1, len(models), 1+mi)
        plt.errorbar(x, y, xerr=xerr, fmt='o')
        plt.xlim([-most, most])
        plt.title(m)
        # Add center spine
        ax = plt.gca()
        p = ax.spines['left'].get_path()
        s = matplotlib.spines.Spine.linear_spine(ax, "right")
        s.register_axis(ax.yaxis)
        s.set_position('zero')
        s.set_color('gray')
        ax.artists.append(s)
        # Wow, that was a lot of code
        if mi > 0:
            plt.yticks([], [])
        else:
            plt.yticks(dfm.index, dfm["factor"])